In [ ]:
! pip install langchain_community

In [ ]:
!pip install fitz chromadb peft

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git


In [ ]:
! pip install frontend

In [5]:
! pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 88.5 MB/s eta 0:00:00:00:0100:01


In [ ]:
!pip install chromadb

In [ ]:
! pip install --upgrade transformers==4.41.0 torch==2.2.1


In [15]:
!pip install accelerate>=0.27.0

In [ ]:
# Kaggle-specific installation (run first)
!pip install -U "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" \
    xformers==0.0.26 \
    "huggingface_hub[cli]" \
    --no-deps
!pip install --force-reinstall torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 \
    --index-url https://download.pytorch.org/whl/cu121


In [ ]:
!pip install unsloth vllm

In [42]:
import os
import numpy as np
# import fitz  # PyMuPDF for PDF text extraction
import torch
import chromadb
from chromadb import Client, Settings
from chromadb.utils import embedding_functions
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
import pymupdf 
# from unsloth import FastLanguageModel
# Function to process PDFs from a folder.
def process_pdfs(pdf_folder):
    documents = []
    text= 'Hi'
    for file in os.listdir(pdf_folder):
        if file.endswith(".pdf"):
            filepath = os.path.join(pdf_folder, file)
            # with fitz.open(filepath) as doc:
            #     text = ""
            #     for i in range(len(doc)):
            #         page = doc.load_page(i)
            #         text += page.get_text("text
            doc= pymupdf.open(filepath)
            for page in doc: # iterate the document pages
                  text += page.get_text() 
                # Split the extracted text into chunks (e.g., 500 characters each)
            chunks = [text[i:i+500] for i in range(0, len(text), 500)]
            documents.extend(chunks)
    return documents

    doc = pymupdf.open("/kaggle/input/cbt-docs/narrative_therapy.pdf") # open a document
    for page in doc: # iterate the document pages
      text += page.get_text() 
    file = open('narrative_therapy.txt', 'w') 
    file.write(text) 
    file.close()

# Enhanced vector database with QIM integration.
class EnhancedTherapyDatabase:
    def __init__(self):
     
        self.client = chromadb.PersistentClient(path=".therapy_db")
        self.ef = embedding_functions.DefaultEmbeddingFunction()
        self.collection = self.client.get_or_create_collection(
            name="therapy_docs",
            embedding_function=self.ef
        )
        self.global_mean = None
        self.global_std = None
        self.bin_counts = {}

    def add_documents(self, documents):
        # Compute embeddings with the default embedding function.
        embeddings = [self.ef([doc])[0] for doc in documents]
        self.collection.add(documents=documents,
                            ids=[str(i) for i in range(len(documents))],
                            embeddings=embeddings)
        # Compute global statistics (mean, std) and bin counts used in QIM.
        self.compute_global_stats(embeddings)

    def compute_global_stats(self, embeddings):
        all_emb = np.array(embeddings)
        self.global_mean = np.mean(all_emb, axis=0)
        self.global_std = np.std(all_emb, axis=0) + 1e-6  # avoid division by zero
        # For each dimension, quantize the embedding to 4 bits (0-15) and count occurrences
        for vec in embeddings:
            quantized = (vec * 15).astype(np.uint8)
            for i, val in enumerate(quantized):
                key = (i, int(val))
                self.bin_counts[key] = self.bin_counts.get(key, 0) + 1

    def qim_score(self, query_embed, doc_embed):
        # Convert embeddings to 4-bit representation
        q_bin = (query_embed * 15).astype(np.uint8)
        d_bin = (doc_embed * 15).astype(np.uint8)
        score = 0.0
        for i in range(len(query_embed)):
            diff = (query_embed[i] - doc_embed[i]) ** 2
            key = (i, int(d_bin[i]))
            N_i = self.bin_counts.get(key, 1)
            score += diff * (N_i ** 2)
        # Normalize score using the average global standard deviation
        score = score / (len(query_embed) * np.mean(self.global_std))
        return score

    def query_qim(self, query_text, n_results=3):
        query_embed = self.ef([query_text])[0]
        all_docs = self.collection.get(include=["embeddings", "documents", "ids"])
        scores = []
        for doc_id, doc_embed, doc_text in zip(all_docs["ids"],
                                                 all_docs["embeddings"],
                                                 all_docs["documents"]):
            # Compute the QIM score between query and document embedding
            score = self.qim_score(np.array(query_embed), np.array(doc_embed))
            scores.append((score, doc_text))
        # Rank documents by descending QIM score (higher scores denote higher influence)
        top_docs = sorted(scores, key=lambda x: x[0], reverse=True)[:n_results]
        return "\n".join([doc for score, doc in top_docs])

# Function to load a quantized, fine-tuned LLM using QLoRA.
def load_llm(model_name):
    # quant_config = BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_quant_type="nf4",
    #     bnb_4bit_compute_dtype=torch.float16,
    #     bnb_4bit_use_double_quant=True
    # )
    # lora_config = LoraConfig(
    #     r=64,                 # attention dimension for LoRA
    #     lora_alpha=16,        # scaling factor
    #     target_modules=["q_proj", "v_proj"],
    #     lora_dropout=0.1,
    #     bias="none",
    #     task_type="CAUSAL_LM"
    # )
    # tokenizer = AutoTokenizer.from_pretrained(model_name, load_in_4bit=True)
    # model = AutoModelForCausalLM.from_pretrained(model_name,
    #                                              quantization_config=quant_config,
    #                                              device_map="auto", trust_remote_code=True)
    # model = get_peft_model(model, lora_config)
    # model.eval()
    # return tokenizer, model
    model, tokenizer = AutoModelForCausalLM.from_pretrained(
        model_name, # Pre-quantized
        max_seq_length = 2048,
        dtype = torch.float16,
        load_in_4bit = True,
        device_map = "auto",
        trust_remote_code= True
    )
    
    # 2. Disable complex features for Kaggle compatibility 
    model.config.pretraining_tp = 1 # Disable tensor parallelism
    model.config.use_cache = False

# Therapy session class that combines context from the QIM database with LLM generation.
class TherapySession:
    def __init__(self, db, tokenizer, model):
        self.db = db
        self.tokenizer = tokenizer
        self.model = model
        self.chat_history = []
    
    def get_context(self, user_input):
        # Retrieve context from the vector store using QIM ranking.
        context = self.db.query_qim(user_input, n_results=3)
        return context

    def generate_response(self, user_input):
        context = self.get_context(user_input)
        # Build a prompt incorporating therapeutic context, conversation history, and current user input.
        prompt = (
            "You are a compassionate therapy assistant for war victims struggling with trauma. "
            "Use the following context extracted from therapeutic literature to guide your answer:\n"
            f"{context}\n\n"
            "Conversation History:\n"
        )
        for turn in self.chat_history:
            prompt += f"User: {turn['user']}\nAssistant: {turn['assistant']}\n"
        prompt += f"User: {user_input}\nAssistant:"

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        output = self.model.generate(**inputs, max_new_tokens=150, do_sample=True, temperature=0.7)
        response = self.tokenizer.decode(output[0], skip_special_tokens=True)
        if "Assistant:" in response:
            response = response.split("Assistant:")[-1].strip()
        self.chat_history.append({"user": user_input, "assistant": response})
        return response




In [ ]:

# Process PDFs from the designated folder (e.g., texts related to therapy for war victims)
pdf_folder = "./therapy_pdfs"  # ensure your PDFs are placed here
documents = process_pdfs('/kaggle/input/cbt-docs')
print("Processed documents from PDFs.")

In [ ]:
# Initialize the enhanced vector DB with QIM scoring.
db = EnhancedTherapyDatabase()
db.add_documents(documents)
print("Added documents to therapy database and computed global statistics.")



In [ ]:
# Load the quantized fine-tuned LLM (the model should be available locally to run under low network conditions).
model_name  = "unsloth/DeepSeek-R1-GGUF"  # replace with your local/fine-tuned model identifier
tokenizer, model = load_llm(model_name)
print("Loaded quantized LLM model.")


In [ ]:

# Start the interactive therapy session.
session = TherapySession(db, tokenizer, model)
print("Therapy assistant: Hello, I'm here to support you. How are you feeling today?")

while True:
    try:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Therapy assistant: Thank you for sharing. Remember that you're always supported.")
            break
        response = session.generate_response(user_input)
        print(f"Therapy assistant: {response}")
    except KeyboardInterrupt:
        print("\nTherapy assistant: Take care. I am always here if you need to talk.")
        break

In [ ]:
# trauma_bot_deepseek.py
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch

# 1. Quantized Model Setup
model_name = "deepseek-ai/deepseek-llm-7b-chat"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16
# )

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    device_map="auto"
)

# 2. Custom PDF Processing
def process_pdfs(pdf_paths):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=50
    )
    
    docs = []
    for path in pdf_paths:
        loader = PyPDFLoader(path)
        docs.extend(loader.load_and_split(text_splitter))
    
    return FAISS.from_documents(docs, HuggingFaceEmbeddings())

# 3. Memory-Enhanced Generation
# memory = ConversationBufferMemory()
vector_store = process_pdfs(["/kaggle/input/cbt-docs/2017-01-08_07-23-35_essays_on_war_traumas_adaptation_and_rehabilitation.pdf", 
                                   "/kaggle/input/cbt-docs/TF-CBT-toolkit-with-color-laminates-1.30.12.pdf",
                                 '/kaggle/input/cbt-docs/Your-Very-Own-TF-CBT-Workbook-Final.pdf',
                                  '/kaggle/input/cbt-docs/cbt.pdf',
                                  '/kaggle/input/cbt-docs/narrative_therapy.pdf'])

def generate_response(question):
    # Retrieve relevant context
    docs = vector_store.similarity_search(question, k=3)
    context = "\n".join([d.page_content for d in docs])
    
    # Memory integration
    # history = memory.load_memory_variables({})['history']
    
    # Trauma-focused prompt template
    # Conversation history:
    # {history}
    prompt = f"""<｜begin▁of▁sentence｜>You are a trauma specialist AI. Use this context:
    {context}
    
    
    
    Current query: {question}
    
    Apply these steps:
    1. Validate emotional state
    2. Suggest CBT technique from manuals
    3. Offer crisis resources if needed
    Response:"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,
        temperature=0.7,
        repetition_penalty=1.1
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    memory.save_context({"input": question}, {"output": response})
    
    return response.split("Response:")[-1].strip()

# Example usage
print(generate_response("How do I handle survivor's guilt after losing family?"))


In [16]:
vector_store = process_pdfs([
                                   "/kaggle/input/cbt-docs/TF-CBT-toolkit-with-color-laminates-1.30.12.pdf",
                                 '/kaggle/input/cbt-docs/Your-Very-Own-TF-CBT-Workbook-Final.pdf'
                                
                            ])

def generate_response(question):
    # Retrieve relevant context
    docs = vector_store.similarity_search(question, k=3)
    context = "\n".join([d.page_content for d in docs])
    
    # Memory integration
    # history = memory.load_memory_variables({})['history']
    
    # Trauma-focused prompt template
    # Conversation history:
    # {history}
    prompt = f"""<｜begin▁of▁sentence｜>You are a trauma specialist AI. Use this context:
    {context}
    
    
    
    Current query: {question}
    
    Apply these steps:
    1. Validate emotional state
    2. Suggest CBT technique from manuals
    3. Offer crisis resources if needed
    Response:"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,
        temperature=0.7,
        repetition_penalty=1.1
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # memory.save_context({"input": question}, {"output": response})
    
    return response.split("Response:")[-1].strip()

# Example usage
print(generate_response("How do I handle survivor's guilt after losing family?"))


<ipython-input-10-ed07debecfcc>:35: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  return FAISS.from_documents(docs, HuggingFaceEmbeddings())
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


It sounds like you may be experiencing feelings of survivor's guilt after losing a loved one. This can be a common response to traumatic events, especially when individuals feel responsible for the outcome or believe they could have done something differently to prevent the loss. To cope with these emotions, please remember that there are several cognitive-behavioral techniques that you can try:

1. Acknowledge and validate your emotions - Recognize that feeling guilty about surviving while others did not is a normal part of grief and trauma. Allow yourself time to process these feelings without judgment.

2. Challenge negative thoughts using Cognitive Restructuring (CR): Survivor’s guilt often involves blaming oneself for the loss. Work through these thoughts by identifying any irrational beliefs and replacing them with more balanced perspectives. For example, consider whether it's reasonable to assume you should have been able to predict or control the situation.

3. Practice self-co